In [1]:
import cv2
import os

from collections import OrderedDict
from ipywidgets import IntSlider, Label, Button, HBox, Output, widgets
from ipycanvas import MultiCanvas, hold_canvas
import math

In [2]:
# image가 저장된 경로 작성
# 이미지가 저장된 그 폴더 경로를 작성해주세요
# 윈도우에서 경로복사 하시면 "C:\Users\user\Desktop\EMBED\annotating\lane_3" 이렇게 복사되는데
# "C:/Users/user/Desktop/EMBED/annotating/lane_3" 이런식으로 역슬래쉬를 그냥 슬래쉬로 변경해주세요
img_foldername = "C:/Users/user/Desktop/EMBED/best_fail/best_fail"

# annotation.txt가 저장될 경로 작성
# 실제 그 경로에 annotation.txt가 저장 되어 있을 필요는 없고
# 밑에 annotating 진행하시면 그 위치에 txt 파일이 생성될겁니다
# 입력예시 ann_filename = 'C:/Users/user/Desktop/EMBED/annotating/annotation.txt'
ann_filename = 'C:/Users/user/Desktop/EMBED/this_is_no.txt'

In [12]:
# 이미지 이름 재정렬 해주는 코드
# 00001.jpg부터 순서대로 저장되도록 이름을 바꿔줍니다

import os
import shutil

def reorder_jpgs(folder_path):
    # Retrieve all jpg files in the folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

    # Sort files based on the numeric value of the filename
    files.sort(key=lambda x: int(x.split('.')[0]))

    # Pad numbers and rename files
    for i, filename in enumerate(files, start=1):
        new_filename = f"{i:05d}.jpg"  # Formats the number with leading zeros
        original_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, new_filename)
        shutil.move(original_path, new_path)  # Rename the file

  # Replace with your folder path
reorder_jpgs(img_foldername)


In [3]:
thickness = 3
y_ratio = 0.5

ann_dict = OrderedDict()

num_frames = len(os.listdir(img_foldername))


height, width = cv2.imread(str(img_foldername + '/{:05d}.jpg'.format(1))).shape[:2]
y_value = int(height * y_ratio)

밑에 코드 실행하고 몇개 정도만 시험삼아 레이블링해서 save lables 눌러보고 txt 파일 잘 저장되는지 확인해 주세요


몇개 정도 순서대로 하시고 중단하셨다가 나중에 다시 실행하시면 자동으로 이어서 레이블링 할 수 있도록 구현했습니다

In [4]:
cur_index = 1
ann_dict = OrderedDict()
try :
    with open(ann_filename, 'r') as f:
        line = f.read()
        print(f"LAST ANNOTATED IMAGE WAS {line.split()[-5]}\n")
        cur_index = int(line.split()[-5][:-4])+1
        print(f"---------------------------START ANNOTATING FROM {str('{:05d}.jpg'.format(cur_index))}---------------------------\n")

except:
    print("---------------------------START ANNOTATING FROM 00001.jpg---------------------------")

def set_image():
    image = cv2.imread(str(img_foldername + '/{:05d}.jpg'.format(cur_index)))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image[y_value-thickness//2:y_value+thickness//2] = (255, 0, 0)

    canvas[0].clear()
    canvas[0].put_image_data(image, 0, 0)
    canvas[0].flush()

    pos = ann_dict.get(str('{:05d}.jpg'.format(cur_index)))
    if pos is not None:
        handle_mouse_down(pos[0], pos[1])

    cur_fname.value = 'Current image: {:s} | '.format(str(img_foldername + '/{:05d}.jpg'.format(cur_index)))

def handle_mouse_move(xpos, ypos):
    with hold_canvas():
        canvas[1].clear()  # Clear the old animation step
        canvas[1].fill_style = "yellow"
        canvas[1].fill_circle(xpos, y_value, 5)  # Draw the new frame

def handle_mouse_down(xpos, ypos):
    with hold_canvas():
        canvas[2].clear()
        canvas[2].fill_style = "green"
        canvas[2].fill_circle(xpos, y_value, 5)  # Draw the new frame

    cur_pos.value = "({:f}, {:f}) ".format(xpos, y_value)
    ann_dict[str('{:05d}.jpg'.format(cur_index))] = (int(out_of_range_chbx.value), int(intersection_chbx.value), xpos, y_value)

def handle_slider_change(change):
    global y_value
    y_value = change.new
    set_image()
    canvas[1].clear()
    canvas[2].clear()

def handle_save_button(b):
    with open(ann_filename, 'a') as f:
        for jpg_name, values in ann_dict.items():
            # f.write("{:s}\t{:d}\t{:d}\t{:d}\t{:d}\n".format(jpg_name, int(values[0]), int(values[1]), int(values[2]), int(values[3])))
            f.write("{:s}\t{:d}\t{:d}\t{:d}\t{:d}\n".format(jpg_name, int(values[0]), int(values[1]), int(values[2]), int(values[3])))

    print("ANNOTATION SAVE SUCCESS")

def handle_prev_button(b):
    global cur_index
    cur_index = max(0, cur_index - 1)
    canvas.clear()
    set_image()
    

def handle_next_button(b):
    global cur_index
    cur_index = min(num_frames - 1, cur_index + 1)
    canvas.clear()
    set_image()


def on_keyboard_event(key, shift_key, ctrl_key, meta_key):
    global cur_index
    if shift_key:
        cur_index = min(num_frames - 1, cur_index +1)
        canvas.clear()
        set_image()
    elif ctrl_key:
        cur_index = max(0, cur_index - 1)
        canvas.clear()
        set_image()



canvas = MultiCanvas(3, width=width, height=height)
cur_fname = Label(value='', disabled=False)
cur_pos = Label(value='', disabled=True)
yslider = IntSlider(description="Y-bar: ", stype={'description_width': 'initial'}, value=y_value, min=1, max=height-2, step=1)

prev_btn = Button(description='Prev', icon='arrow-left')
next_btn = Button(description='Next', icon='arrow-right')
save_btn = Button(description='Save labels', icon='check')

out_of_range_chbx = widgets.Checkbox(value=False, description='Out of range', disabled=False, indent=False)
intersection_chbx = widgets.Checkbox(value=False, description='Intersection', disabled=False, indent=False)



set_image()
canvas.on_mouse_move(handle_mouse_move)
canvas.on_mouse_down(handle_mouse_down)
yslider.observe(handle_slider_change, names='value')

prev_btn.on_click(handle_prev_button)
next_btn.on_click(handle_next_button)
save_btn.on_click(handle_save_button)

canvas.on_key_down(on_keyboard_event)

display(canvas, HBox([cur_fname, cur_pos, yslider]), HBox([out_of_range_chbx, intersection_chbx]), HBox([prev_btn, next_btn, save_btn]))


---------------------------START ANNOTATING FROM 00001.jpg---------------------------


MultiCanvas(height=360, width=640)